In [9]:
#Linear Regression Newtons Method
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
data = pd.read_csv('../Assignment_data/Data_Q1/housing.csv') #Loading the data from the housing.csv file
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [10]:
from sklearn.model_selection import train_test_split  #Splitting the data into test and train data.

train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)


In [11]:
import pandas as pd
from sklearn.preprocessing import Imputer

train_data = train_set.drop(["median_house_value","ocean_proximity"],axis=1) #all the data set after droping 2 col
train_labels = pd.DataFrame(train_set["median_house_value"]) #a matrix of y(median_house_value)
#Data Cleaning of training data

imputer = Imputer(strategy="median")
imputer.fit(train_data)
X = imputer.transform(train_data)
leny = len(train_labels)
imputer.fit(train_labels)
Y = imputer.transform(train_labels)
X1 = np.c_[np.ones((leny,1)),X]
Y1 = pd.DataFrame(Y)

#Data Cleaning of test data

test_data = test_set.drop(["median_house_value","ocean_proximity"],axis=1)
imputer.fit(test_data)#Cleaning the test data
test_labels = pd.DataFrame(test_set["median_house_value"])
X2 = imputer.transform(test_data)
leny1 = len(test_labels)
imputer.fit(test_labels)
Y2 = imputer.transform(test_labels)
X3 = np.c_[np.ones((leny1,1)),X2]
Y3 = pd.DataFrame(Y2)



In [12]:
iterations = 100

def cost_function(X, y, theta):
    #len of train_labels
    m = len(y) 
    # Calculate the cost with the given parameters
    J = np.sum((X.dot(theta)-y)**2)/m
    
    return J

In [13]:
from numpy.linalg import inv

dim = len(X1[0])  #Dimension of the data
theta = np.zeros((dim,1))

# Hessian can be calculated from below formula
# hessian = derivative of gardient(MSE(theta)) = derivative((2/m)X.T.(X.dot(theta)-y)) = (2/m)*(X.T.dot(X))

def compute_hessian(X,y,theta):  #Function which return the hessian matrix.
    m = len(y)
    hessian = (2*((X.T).dot(X)))/m      
    return hessian
    
def newtons_method(X, y,theta,iterations): #Function which returns theta and the theta_history.

    theta_history = []
    theta_history.append(theta)
    #alpha = compute_hessian(X,y,theta)
    m = len(y)
    
    for iteration in range(iterations):
        hypothesis = X.dot(theta)
        error = hypothesis-y
        gradient = (2 * (X.T).dot(error))/m #derivative of cost function
        alpha = compute_hessian(X,y,theta) #computing hessian matrix
        theta = theta - (inv(alpha).dot(gradient))#Theta_new=Theta_old-hessian*gradient
        theta_history.append(theta)
        
    return theta,theta_history

In [14]:
%%time
W,theta_history = newtons_method(X1,Y1,theta,iterations)

CPU times: user 168 ms, sys: 4 ms, total: 172 ms
Wall time: 85.2 ms


In [15]:
#Parameters
W

array([[-3.57822423e+06],
       [-4.26323917e+04],
       [-4.24500719e+04],
       [ 1.18280965e+03],
       [-8.18797708e+00],
       [ 1.16260128e+02],
       [-3.84922131e+01],
       [ 4.63425720e+01],
       [ 4.05384044e+04]])

In [16]:
from sklearn.metrics import mean_squared_error #finding the mean square error.
def RMSE(X,W,Y) :
    housing_predictions = X.dot(W)
    lin_mse = mean_squared_error(Y,housing_predictions)
    lin_rmse = np.sqrt(lin_mse)
    return lin_rmse

RMSE(X3,W,Y3)

71140.1732833627